In [3]:
from openai import OpenAI
import json

BASE_URL = "http://0.0.0.0:8080/v1"
API_KEY = "sk-xxx"
MODEL_ID = "functionary-small-v2.5-auto-tokenizer"

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

# Baseline

Test out a question without using tools as a baseline

In [4]:
response = client.chat.completions.create(
    model=MODEL_ID,
    messages=[
        {
            "content": "You are a helpful assistant.",
            "role": "system"
        },
        {
            "content": "What is the capital of France?",
            "role": "user"
        }
    ],
)
for choice in response.choices:
    print(choice.message.content)

The capital of France is Paris.


In [5]:
def chat_completion_request(messages, tools=None, model=MODEL_ID):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            # tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [6]:
messages = [
    {"role": "user", "content": "what's the weather like in Hanoi?"}
]
tools = [ # For functionary-7b-v2 we use "tools"; for functionary-7b-v1.4 we use "functions" = [{"name": "get_current_weather", "description":..., "parameters": ....}]
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g., San Francisco, CA",
                    }
                },
                "required": ["location"]
            }
        }
    }
]

In [7]:
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response

ChatCompletion(id='chatcmpl-963762bc-155f-4f9d-8140-2082a36aa81a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<|reserved_special_token_249|>weather\n{"location": "Hanoi, Vietnam"}', role='assistant', function_call=None, tool_calls=None))], created=1720034279, model='functionary-small-v2.5-auto-tokenizer', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=15, total_tokens=27))

In [8]:
import yaml

print(yaml.dump(chat_response.model_dump(), sort_keys=False, explicit_start=True))


---
id: chatcmpl-963762bc-155f-4f9d-8140-2082a36aa81a
choices:
- finish_reason: stop
  index: 0
  logprobs: null
  message:
    content: '<|reserved_special_token_249|>weather

      {"location": "Hanoi, Vietnam"}'
    role: assistant
    function_call: null
    tool_calls: null
created: 1720034279
model: functionary-small-v2.5-auto-tokenizer
object: chat.completion
service_tier: null
system_fingerprint: null
usage:
  completion_tokens: 12
  prompt_tokens: 15
  total_tokens: 27

